In [1]:
!which python3
!python3  --version

/usr/bin/python3
Python 3.12.3


In [273]:
import sys
!{sys.executable} -m pip install bitsandbytes==0.44.0 accelerate==1.1.1 peft==0.14.0 transformers==4.46.3 torch==2.5.1 datasets==3.6.0 evaluate rouge_score aiohttp
# Need old datasets version due to incompatible dataset format (.py script)
# Need old transformers due to `AttributeError: 'DynamicCache' object has no attribute 'seen_tokens'` on new versions
# Need old accelerate to avoid issues with quantization failing

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import AutoTokenizer
from aiohttp import ClientTimeout

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_from_disk

tokenized_5k_sp = load_from_disk("/workspace/tokenized_5k_sp")

In [11]:
import json
# print(json.dumps(tokenized_5k_sp['train'][:5], indent=1))
print(len(tokenized_5k_sp["train"])) # 52945k entries

52945


In [6]:
prefix = "generate a summary that will serve as abstract for the following scientific paper:\n\n"

def preprocess(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=128000, truncation=True) # max_length here could be reduced to 5000 and truncation disabled

    labels = tokenizer(examples["abstract"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
from datasets import load_from_disk
import os

# Define the path in Google Drive
drive_path = "/content/drive/MyDrive/Colab Notebooks/abstract-gen/tokenized_sp"

if os.path.exists(drive_path):
  tokenized_sp = load_from_disk(drive_path)
else:
  tokenized_sp = sp.map(preprocess, batched=True)
  tokenized_sp.save_to_disk(drive_path)

Loading dataset from disk:   0%|          | 0/35 [00:00<?, ?it/s]

In [13]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred # Each pred in preds is an array of token ids
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count.nonzero(pred != tokenizer.pad_token_id) for pred in preds]  # Generates a boolean array for each pred (!= in np works element-wise in this case)
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# import sys
# !{sys.executable} -m pip install peft==0.44.0

In [ ]:
# import importlib

# import numpy
# importlib.reload(numpy)

In [175]:
import torch
from transformers import pipeline, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16
)

# Quantized model
model = AutoModelForCausalLM.from_pretrained(
        checkpoint,
        # return_dict=True,
        # low_cpu_mem_usage=True,
        # torch_dtype="auto",
        # device_map="auto",
        # trust_remote_code=True,
        quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
summ = pipeline("text-generation", model=model, tokenizer=tokenizer)

gen_args = {
  "max_new_tokens": 256,
  "return_full_text": False,
  "temperature": 0.0,
  "do_sample": False,
}

# summ("generate a summary that will serve as abstract for the following scientific paper:\n\n" + sp['train'][0]['article'], **gen_args)
print(summ("generate a summary:\n\n" + "Microsoft has joined the competitive landscape of large language models alongside Meta AI with the introduction of the Phi-3.5 series. This series includes a small language model, a vision language model, and employs a Mixture-of-Experts approach to achieve top-tier performance. In this tutorial, we will explore the Microsoft Phi-3.5 family of models. We will load the Phi-3.5-mini-instruct model and fine-tune it to classify e-commerce products based on their text descriptions. In the final steps, we will demonstrate how to merge the LoRA (Low-Rank Adaptation) with the base model and push it to Hugging Face. This will enable efficient cloud deployment, making the model accessible for various applications.", **gen_args))

In [176]:
from peft import LoraConfig, TaskType, get_peft_model

import peft
print("peft:", peft.__version__)

from peft import prepare_model_for_kbit_training

peft_config = LoraConfig(
    r=64,
    lora_alpha=128,
    task_type=TaskType.CAUSAL_LM,
    target_modules=["o_proj", "qkv_proj"] # Check model arch to see how it has these two layers
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

peft: 0.14.0
trainable params: 37,748,736 || all params: 3,858,828,288 || trainable%: 0.9782


In [185]:
len(train_sample[0]["input_ids"])

188

# Encoder-decoder (BART)

In [15]:
from datasets import load_from_disk
from aiohttp import ClientTimeout
import os

drive_path_raw_dataset = "/workspace/scientific_papers_hf"

if os.path.exists(drive_path_raw_dataset):
  sp = load_from_disk(drive_path_raw_dataset)
else:
  notebook_login()
  sp = load_dataset("armanc/scientific_papers", "arxiv", trust_remote_code=True, storage_options={'client_kwargs': {'timeout': ClientTimeout(total=3600)}})
  sp.save_to_disk(drive_path_raw_dataset)

In [4]:
sp_small = sp.filter(lambda ex: len(ex["article"]) > 1000 and len(ex["article"]) < 15000) 

Filter:   0%|          | 0/203037 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6436 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [3]:
checkpoint = "allenai/led-base-16384"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
tokenizer.convert_ids_to_tokens(tokenizer("I loved reading the Hunger Games!").input_ids)


['<s>', 'I', 'Ġloved', 'Ġreading', 'Ġthe', 'ĠHunger', 'ĠGames', '!', '</s>']

In [4]:
max_input_length = 5120
max_target_length = 256

def preprocess(examples):
    model_inputs = tokenizer(examples["article"], max_length=max_input_length, truncation=True)

    labels = tokenizer(examples["abstract"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [4]:
from datasets import load_from_disk
import os

path = "/workspace/tokenized_sp_small"

if os.path.exists(path):
  tokenized_sp_small = load_from_disk(path)
else:
  tokenized_sp_small = sp_small.map(preprocess, batched=True)
  tokenized_sp_small.save_to_disk(path)

In [9]:
for i in range(5):
    print(len(tokenized_sp_small["train"][i]["input_ids"]))

4599
2291
2832
959
1740


In [17]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, logging

logging.set_verbosity_error()

batch_size = 6
epochs = 2

# KIND OF UNUSED with accelerate

training_args = Seq2SeqTrainingArguments(
    output_dir="led-abstract-gen",
    eval_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=3, # Only save up to 3 checkpoints (hard drive space savings)
    num_train_epochs=epochs,
    predict_with_generate=True, # Evaluate model based on generated summaries
    fp16=True,
    # optim="adafactor",
    push_to_hub=True, #
)

In [256]:
# UNUSED with accelerate

import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred # Each pred in preds is an array of token ids
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) # Decode generated summaries into text
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # Can't decode -100 in labels so we replace them
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) # Decode reference summaries into text

    # ROUGE expects a new line after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # prediction_lens = [np.count.nonzero(pred != tokenizer.pad_token_id) for pred in preds]  # Generates a boolean array for each pred (!= in np works element-wise in this case)
    # result["gen_len"] = np.mean(prediction_lens)

    result = {key: value.mid.fmeasure * 100 for key, value in result.items()} # Extract median scoress

    return {k: round(v, 4) for k, v in result.items()}

In [29]:
from transformers import DataCollatorForSeq2Seq

# Also shifts the the labels to the right by one during decoding so that the decoder
# only sees the previous labels and can't cheat. This is necessary for encoder-decoder models
# The pad_to_multiple_of is used to prevent annoying warnings at training time
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, pad_to_multiple_of=model.config.attention_window[0])

In [27]:
# We need to remove the string columns (only leave the vector ones) otherwise we will get an error in the data collator
# More info: https://huggingface.co/learn/llm-course/chapter7/5#fine-tuning-mt5-with-the-trainer-api
tokenized_sp_small = tokenized_sp_small.remove_columns(sp["train"].column_names) # Equivalent to ["article", "abstract", "section_names"]

ValueError: Column name ['section_names', 'abstract', 'article'] not in the dataset. Current columns in the dataset: ['input_ids', 'attention_mask', 'labels']

In [28]:
len(data_collator([tokenized_sp_small["train"][i] for i in range(2)])["input_ids"][0])

5120

In [267]:
# UNUSED with accelerate

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sp_small["train"],
    eval_dataset=tokenized_sp_small["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [4]:
import torch
from transformers import pipeline, AutoModelForSeq2SeqLM, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_storage=torch.bfloat16
# )

# Quantized model
model = AutoModelForSeq2SeqLM.from_pretrained(
        checkpoint,
        # return_dict=True,
        # low_cpu_mem_usage=True,
        # torch_dtype="auto",
        # device_map="auto",
        # trust_remote_code=True,
        # quantization_config=quantization_config
)

In [5]:
for name, module in model.named_modules():
    print(name, type(module))

 <class 'transformers.models.led.modeling_led.LEDForConditionalGeneration'>
led <class 'transformers.models.led.modeling_led.LEDModel'>
led.shared <class 'torch.nn.modules.sparse.Embedding'>
led.encoder <class 'transformers.models.led.modeling_led.LEDEncoder'>
led.encoder.embed_positions <class 'transformers.models.led.modeling_led.LEDLearnedPositionalEmbedding'>
led.encoder.layers <class 'torch.nn.modules.container.ModuleList'>
led.encoder.layers.0 <class 'transformers.models.led.modeling_led.LEDEncoderLayer'>
led.encoder.layers.0.self_attn <class 'transformers.models.led.modeling_led.LEDEncoderAttention'>
led.encoder.layers.0.self_attn.longformer_self_attn <class 'transformers.models.led.modeling_led.LEDEncoderSelfAttention'>
led.encoder.layers.0.self_attn.longformer_self_attn.query <class 'torch.nn.modules.linear.Linear'>
led.encoder.layers.0.self_attn.longformer_self_attn.key <class 'torch.nn.modules.linear.Linear'>
led.encoder.layers.0.self_attn.longformer_self_attn.value <class '

In [13]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [10]:
import nltk

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [6]:
tokenized_sp_small.set_format("torch")

In [25]:
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = epochs * num_update_steps_per_epoch

In [61]:
from transformers import get_scheduler

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [21]:
# There is a notable reduction in VRAM usage by using adafactor vs adam
from transformers.optimization import Adafactor, AdafactorSchedule

optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=1e-3) # Not sure about these settings
lr_scheduler = AdafactorSchedule(optimizer)

In [22]:
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader

batch_size = training_args.per_device_train_batch_size
gradient_accumulation_steps = training_args.gradient_accumulation_steps

train_dataloader = DataLoader(tokenized_sp_small["train"], batch_size=batch_size, collate_fn=data_collator, shuffle=True)

eval_dataloader = DataLoader(tokenized_sp_small["validation"], batch_size=batch_size, collate_fn=data_collator)

if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

accelerator = Accelerator(mixed_precision="fp16", gradient_accumulation_steps=gradient_accumulation_steps)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

# model.train()

# for step, batch in enumerate(dataloader, start=1):
#     loss = model(**batch).loss
#     loss = loss / training_args.gradient_accumulation_steps
#     accelerator.backward(loss)
#     if step % training_args.gradient_accumulation_steps == 0:
#         optimizer.step()
#         optimizer.zero_grad()

In [18]:
from huggingface_hub import get_full_repo_name

model_name = "led-abstract-gen"
repo_name = get_full_repo_name(model_name)
repo_name

'miguelgrc/led-abstract-gen'

In [26]:
from huggingface_hub import create_repo
create_repo(repo_name)

RepoUrl('https://huggingface.co/miguelgrc/led-abstract-gen', endpoint='https://huggingface.co', repo_type='model', repo_id='miguelgrc/led-abstract-gen')

In [29]:
# Just to debug if needed

model.cpu()
batch = next(iter(train_dataloader))
batch = {k: v.cpu() for k, v in batch.items()}

with torch.no_grad():
    out = model(**batch)
print(out.loss)

KeyboardInterrupt: 

KeyboardInterrupt: 

In [21]:
len(train_dataloader)

8987

In [36]:
len(train_dataloader) * epochs
# == (len(tokenized_sp_small["train"]) // batch_size) * epochs

11984

In [7]:
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    print(f"Epoch {epoch}:", result)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        unwrapped_model.push_to_hub( # Is it model.push_to_hub??
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

NameError: name 'num_training_steps' is not defined

Next steps:
- LoRA
- Quantization 

In [19]:
sp = load_from_disk("/workspace/scientific_papers_hf")
tokenized_sp_small = load_from_disk("/workspace/tokenized_sp_small")

tss = tokenized_sp_small["train"].select(range(100))

tss

Dataset({
    features: ['article', 'abstract', 'section_names', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [1]:
import torch
from accelerate import Accelerator, DistributedDataParallelKwargs
from torch.utils.data.dataloader import DataLoader
from transformers.optimization import Adafactor, AdafactorSchedule
from transformers import pipeline, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from transformers import DataCollatorForSeq2Seq
from datasets import load_from_disk
from aiohttp import ClientTimeout
import os
from tqdm.auto import tqdm
# from accelerate.utils import tqdm
from transformers import AutoTokenizer
import time
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import pipeline, AutoModelForCausalLM, BitsAndBytesConfig
import numpy as np
import rouge_score
import evaluate
import nltk
nltk.download('punkt_tab')


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels
    

def train_ddp_accelerate():

    output_dir = "led-abstract-gen"
    max_target_length = 256

    # Load tokenizer
    
    checkpoint = "allenai/led-base-16384"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    # Load dataset
    
    sp = load_from_disk("/workspace/scientific_papers_hf")
    tokenized_sp_small = load_from_disk("/workspace/tokenized_sp_small")

    # 3 is good for fine-tuning, 6 is good for LoRA, 8 is good for quantization, 8 is good for QLoRA
    # Note: performance is worse when fine-tuning quantized models than full models, 
    #   we should use QLoRA in that case. Even then, it will be slower than plain LoRA
    #   (it is due to extra computation for quant/dequant and custom, less efficient kernels)
    batch_size = 6 # I needed to reduce it from 6 for multi-gpu. I guess there is some overhead.
    epochs = 4

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, pad_to_multiple_of=1024)
    tokenized_sp_small = tokenized_sp_small.remove_columns(sp["train"].column_names)
    
    train_dataloader = DataLoader(tokenized_sp_small["train"], batch_size=batch_size, collate_fn=data_collator, shuffle=True)
    
    eval_dataloader = DataLoader(tokenized_sp_small["validation"], batch_size=batch_size, collate_fn=data_collator)

    # Accelerator

    # ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
    accelerator = Accelerator(
                              mixed_precision="fp16", # remove for quantized models
                              # kwargs_handlers=[ddp_kwargs], # remove for LoRA
                              # gradient_accumulation_steps=gradient_accumulation_steps
                             )

    # Quantize and build model

    # quantization_config = BitsAndBytesConfig(load_in_8bit=True)
    
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, 
                                                  # quantization_config=quantization_config,
                                                  # device_map={"": accelerator.local_process_index} # needed for quant?
                                                 )

    # model = prepare_model_for_kbit_training(model)

    # LoRA

    lora_config = LoraConfig(
        r=64,
        lora_alpha=128,
        target_modules=["q_proj", "v_proj", "out_proj", "k_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.SEQ_2_SEQ_LM,
    )
    
    model = get_peft_model(model, lora_config)

    if accelerator.is_main_process:
        model.print_trainable_parameters()

    # Build optimizer
    
    optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=1e-3) # Not sure about these settings
    # lr_scheduler = AdafactorSchedule(optimizer) # Doesn't work well, seems to be decaying the lr too quickly and unstabilizing it

    # Prepare

    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

    # Configure evaluator

    rouge_score = evaluate.load("rouge")

    # Progress bar
    
    # num_update_steps_per_epoch = len(train_dataloader)
    # num_training_steps = epochs * num_update_steps_per_epoch
    
    # progress_bar = tqdm(range(num_training_steps), disable=not accelerator.is_local_main_process)

    # Train

    for epoch in range(epochs):
        # Training
        model.train()
        start_time = time.time()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss # Cross-entropy by default for seq2seq
            accelerator.backward(loss)
    
            optimizer.step()
            # lr_scheduler.step()
            optimizer.zero_grad()
            if accelerator.is_main_process and step % 10 == 0:
                elapsed = time.time()-start_time
                el_min, el_sec = divmod(int(elapsed), 60)
                estimated_total = elapsed / (step + 1) * len(train_dataloader)
                rem_min, rem_sec = divmod(int(estimated_total - elapsed), 60)
                print(
                    f"{step + 1}/{len(train_dataloader)} | epoch {epoch + 1}/{epochs} | "
                    f"elapsed: {el_min}m{el_sec}s | per example: {elapsed/(step+1)/batch_size:.2f} | "
                    f"remaining: {rem_min}m{rem_sec}s | loss: {loss.item():.2f}"
                )
    
        # Evaluation
        model.eval()
        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                generated_tokens = accelerator.unwrap_model(model).generate(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    max_new_tokens=max_target_length
                )
    
                generated_tokens = accelerator.pad_across_processes(
                    generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
                )
                labels = batch["labels"]
    
                # If we did not pad to max length, we need to pad the labels too
                labels = accelerator.pad_across_processes(
                    batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
                )
    
                generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
                labels = accelerator.gather(labels).cpu().numpy()
    
                # Replace -100 in the labels as we can't decode them
                labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
                if isinstance(generated_tokens, tuple):
                    generated_tokens = generated_tokens[0]
                decoded_preds = tokenizer.batch_decode(
                    generated_tokens, skip_special_tokens=True
                )
                decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
                decoded_preds, decoded_labels = postprocess_text(
                    decoded_preds, decoded_labels
                )
    
                rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)
    
        # Compute metrics
        if accelerator.is_main_process:
            result = rouge_score.compute()
            # Extract the median ROUGE scores
            result = {k: round(float(v) * 100, 4) for k, v in result.items()}
            print(f"Epoch {epoch}:", result)
    
        # Save and upload
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save) # Run in all processes as model is sharded
        if accelerator.is_main_process:
            tokenizer.save_pretrained(output_dir) # Run only on main process as tokenizer is common
            unwrapped_model.push_to_hub(
                repo_id="miguelgrc/led-abstract-gen",
                commit_message=f"Training in progress epoch {epoch + 1}", 
                blocking=False)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
# To debug errors in one GPU
train_ddp_accelerate()

trainable params: 4,718,592 || all params: 166,563,072 || trainable%: 2.8329
1/34 (epoch 1/2) (elapsed: 0.0m1.244530439376831s) (per example: 0.21) (remaining: 0m41s) (loss: 3.648803234100342
11/34 (epoch 1/2) (elapsed: 0.0m10.207772254943848s) (per example: 0.15) (remaining: 0m21s) (loss: 5.167680740356445
21/34 (epoch 1/2) (elapsed: 0.0m18.382553815841675s) (per example: 0.15) (remaining: 0m11s) (loss: 4.244979381561279
31/34 (epoch 1/2) (elapsed: 0.0m26.86331820487976s) (per example: 0.14) (remaining: 0m2s) (loss: 4.5241475105285645
Epoch 0: {'rouge1': 21.5082, 'rouge2': 5.2163, 'rougeL': 14.5929, 'rougeLsum': 18.8433}


adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

1/34 (epoch 2/2) (elapsed: 0.0m0.8697819709777832s) (per example: 0.14) (remaining: 0m28s) (loss: 4.71817684173584
11/34 (epoch 2/2) (elapsed: 0.0m9.783159255981445s) (per example: 0.15) (remaining: 0m20s) (loss: 4.114262104034424
21/34 (epoch 2/2) (elapsed: 0.0m17.93355369567871s) (per example: 0.14) (remaining: 0m11s) (loss: 3.8215792179107666
31/34 (epoch 2/2) (elapsed: 0.0m26.09733819961548s) (per example: 0.14) (remaining: 0m2s) (loss: 4.694792747497559
Epoch 1: {'rouge1': 21.5082, 'rouge2': 5.2163, 'rougeL': 14.5929, 'rougeLsum': 18.8433}


No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
from accelerate import notebook_launcher

notebook_launcher(train_ddp_accelerate, args=(), num_processes=2)

Launching training on 2 GPUs.
trainable params: 4,718,592 || all params: 166,563,072 || trainable%: 2.8329
1/2996 | epoch 1/4) | elapsed: 0m1s | per example: 0.20 | remaining: 60m47s | loss: 4.80
11/2996 | epoch 1/4) | elapsed: 0m10s | per example: 0.16 | remaining: 47m1s | loss: 2.87


Fine-tuning: 0.35s per example
LoRA: 0.15 per example
8-bit quant: 0.69s per example
QLoRA: 0.74s per example

Advantages of LoRA: less memory, allowing for bigger batch sizes and therefore more speed. Also, the trained model is easier to store (the relatively small adapters).

# GPU monitoring

In [67]:
import torch

total_bytes = 0

for var_name, var_val in list(globals().items()):  # <-- make a list copy
    if torch.is_tensor(var_val) and var_val.is_cuda:
        size_bytes = var_val.element_size() * var_val.nelement()
        total_bytes += size_bytes
        print(f"{var_name}: {size_bytes/1024/1024:.2f} MB ({tuple(var_val.shape)})")
    elif hasattr(var_val, 'parameters'):
        # For models, sum all parameters on CUDA
        model_bytes = 0
        for p in var_val.parameters():
            if p.is_cuda:
                model_bytes += p.element_size() * p.nelement()
        if model_bytes > 0:
            total_bytes += model_bytes
            print(f"{var_name} (model): {model_bytes/1024/1024:.2f} MB")

print(f"\nTotal GPU memory used by variables: {total_bytes/1024/1024:.2f} MB")

loss: 0.00 MB (())
p: 0.00 MB ((768,))
model (model): 617.39 MB

Total GPU memory used by variables: 617.39 MB


In [68]:
import torch, gc
with torch.no_grad():
    torch.cuda.empty_cache()
torch.cuda.ipc_collect()
print(torch.cuda.memory_summary())
gc.collect()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 3            |        cudaMalloc retries: 8         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  31117 MiB |  31501 MiB | 449855 MiB | 418738 MiB |
|       from large pool |  31110 MiB |  31496 MiB | 449621 MiB | 418510 MiB |
|       from small pool |      6 MiB |      7 MiB |    234 MiB |    227 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  31117 MiB |  31501 MiB | 449855 MiB | 418738 MiB |
|       from large pool |  31110 MiB |  31496 MiB | 449621 MiB |

7894

In [ ]:
# model.to('cuda')
# # model.cpu()

In [42]:
del model

In [ ]:
!nvidia-smi

In [53]:
!nvidia-smi --query-compute-apps=pid --format=csv,noheader

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


1578152


In [55]:
!ps -fp 1578152

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


UID          PID    PPID  C STIME TTY          TIME CMD


In [74]:
# Clear VRAM

# !{sys.executable} -m pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 73.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 97.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [numba]32m2/3 [numba]te]


ImportError: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

In [32]:
!{sys.executable} -m pip install nvidia-ml-py3

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'nvidia-ml-py3' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'nvidia-ml-py3'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19208 sha256=40c61b65341140e188272b1cd22a94917c95e4ee1793202b4837209912523c22
  Stored in directory: /root/.cache/pip/wheels/6e/65/79/33dee66cba26e8204801916dfee7481bccfd22905ebb841fe5
Successfully built nvidia-ml-py3


In [64]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [65]:
print_gpu_utilization()

GPU memory occupied: 29565 MB.


In [46]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [121]:
import torch
from torch import nn

# Start recording memory snapshot history
torch.cuda.memory._record_memory_history(max_entries=100000)

model = nn.Linear(10_000, 50_000, device ="cuda")
for _ in range(3):
    inputs = torch.randn(5_000, 10_000, device="cuda")
    outputs = model(inputs)

# Dump memory snapshot history to a file and stop recording
torch.cuda.memory._dump_snapshot("profile.pkl")
torch.cuda.memory._record_memory_history(enabled=None)
